In [1]:
from pyspark.sql.types import *


path = '/FileStore/tables/fhyjgcr41467271069254/wp_posts.csv'
path= '/FileStore/tables/443875ud1470308850012/wp_posts.csv'
path='/FileStore/tables/8904hajl1470383389657/wp_posts4.csv'
scheme = None
        
if scheme == None:
    inferS = True
    
customSchema = StructType([
  StructField('id', StringType(), True),
  StructField('author_id', StringType(), True),
  StructField('post_date', StringType(), True),
  StructField('post_date_gmt', StringType(), True),
  StructField('post_content', StringType(), True),
  StructField('post_title', StringType(), True),
  StructField('post_category', StringType(), True),
  StructField('post_excerpt', StringType(), True),
  StructField('post_status', StringType(), True),
  StructField('comment_status', StringType(), True),
  StructField('ping_status', StringType(), True),
  StructField('post_password', StringType(), True),
  StructField('post_name', StringType(), True),
  StructField('to_ping', StringType(), True),
  StructField('pinged', StringType(), True),
  StructField('post_modifed', StringType(), True),    
  StructField('post_modifed_gmt', StringType(), True),
  StructField('post_content_filtered', StringType(), True),
  StructField('post_parent', StringType(), True),
  StructField('guid', StringType(), True),    
  StructField('menu_ordered', StringType(), True),    
  StructField('post_type', StringType(), True),
  StructField('post_mime_type', StringType(), True),
  StructField('comment_count', StringType(), True)
  ])

    
allData = sqlContext.read.format('com.databricks.spark.csv').options(parserLib='univocity', header='false', delimiter=';', quote='"').schema (customSchema).load(path, inferSchema=False)




In [2]:
from pyspark.sql.functions import year
from datetime import datetime

def setYear ( d ):
  try:
    dtDate = datetime.strptime(d, "%Y-%m-%d %H:%M:%S").year
  except:
    dtDate = 'None'
  return dtDate

setYearUDF = udf(setYear, StringType())

posts = allData.select ('*').where ("post_type = 'post'").withColumn ('post_year', setYearUDF('post_date'))
pages = allData.select ('*').where ("post_type = 'page'")

curYear = '2016'
yearPosts =  posts.select ('*').where ("post_year='" + curYear +"'").cache ()
notValid = posts.select ('*').where ("post_year='None'")

print 'we have %s posts and %s pages. %s in the selected year (%s)' % ( posts.count(), pages.count(), yearPosts.count(), curYear )
print 'not valid %s' % (notValid.count())



In [3]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
import re
import lxml.html

def strip_markdown(x):
  try:
    return re.sub("<.*?>", "", x)
  except:
    return 'none'
    



removeTagsUDF = udf (strip_markdown, StringType())  

def removePunctuation(column):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        column (Column): A Column containing a sentence.

    Returns:
        Column: A Column named 'sentence' with clean-up operations applied.
    """
    #return re.sub( '[^a-z0-9 ]', "", text.lower().strip())
    #return trim (lower (column))
    return  (regexp_replace (trim (lower(column)),'[^a-z0-9 áéíóúñ ]', ' ' )).alias ('sentence')

  
  
  

In [4]:
from pyspark.sql.functions import split, explode

content = yearPosts.withColumn ('post_content', removeTagsUDF('post_content')).select(removePunctuation(col('post_content'))).cache()

words = (content
                .select(explode (split ('sentence', ' ')).alias ('word'))
               ).where ("word<>''")



In [5]:
words.show()


In [6]:
from pyspark.sql.functions import desc
topWords = words.groupBy ('word').count().orderBy( desc ('count') ).cache()


print topWords.show()

print 'total words: %s' % (topWords.count())

In [7]:
path = '/FileStore/tables/5a7qt6081470299078502/stopwords.txt'

stopWords = sqlContext.read.format('com.databricks.spark.csv').options(parserLib='univocity', header='false', delimiter=';').load(path, inferSchema=True)
cleanWords = topWords.join(stopWords,topWords.word == stopWords.C0, 'outer').where("C0 is null").orderBy( desc('count')).drop ('C0')

print 'Total words before clean %s, num stop words %s, total words affter clean %s' % (topWords.count(), stopWords.count(), cleanWords.count())
print cleanWords.show(200)



In [8]:
cleanWords.where("word='días'").show()

In [9]:

df = sqlContext.createDataFrame(
    [(1, "a", 23.0), (3, "B", -23.0),(8, "D", -2.0),(16, "J", -3.0)], ("x1", "x2", "x3"))

df.show()

df.registerTempTable('principal')

df2 = sqlContext.sql ('SELECT ROW_NUMBER() OVER () AS rn, x1,x2,x3 FROM principal')
df3 = sqlContext.sql ('SELECT ROW_NUMBER() OVER ( ORDER BY x2 ) AS rn, x1,x2,x3 FROM principal')
df2.show ()
df3.show ()


In [11]:
df3.orderBy('x2','x3').show()
sqlContext.sql ('SELECT * FROM principal ORDER BY x2,x3').show()


In [12]:
%run "/Users/suscripcion@bluethinking.com/functions"

In [13]:
print suma(4,6)

In [14]:
suma(4,3)